# Modelling Selected Parameters at 0:00 + Day Phase

## Model Selection

In [1]:
# Importing the libraries
import os
import pandas as pd

## Data Preprocessing

In [2]:
# Load and preprocess the train data 
from pipelines import preprocessing_pipeline

data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
train_data = pd.read_csv(data_file, index_col=0, low_memory=False)
train_data = preprocessing_pipeline.fit_transform(train_data)
train_data.head()

,day_phase,bg-2:00,bg-1:55,bg-1:50,bg-1:45,bg-1:40,bg-1:35,bg-1:30,bg-1:25,bg-1:20,...,cals-0:40,cals-0:35,cals-0:30,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,
p01_0,morning,17.2,17.533333,17.866667,18.2,18.266667,18.333333,18.4,18.266667,18.133333,...,0.96,0.96,0.96,0.96,0.96,0.96,0.96,0.96,4.8,13.4
p01_1,morning,18.2,18.266667,18.333333,18.4,18.266667,18.133333,18.0,17.766667,17.533333,...,0.96,0.96,0.96,0.96,0.96,0.96,0.96,0.96,4.8,12.8
p01_2,morning,18.4,18.266667,18.133333,18.0,17.766667,17.533333,17.3,17.366667,17.433333,...,0.96,0.96,0.96,0.96,0.96,0.96,0.96,0.96,4.8,15.5
p01_3,morning,18.0,17.766667,17.533333,17.3,17.366667,17.433333,17.5,17.433333,17.366667,...,0.96,0.96,0.96,0.96,0.96,0.96,0.96,0.96,4.8,14.8
p01_4,morning,17.3,17.366667,17.433333,17.5,17.433333,17.366667,17.3,16.933333,16.566667,...,0.96,0.96,0.96,0.96,0.96,0.96,0.96,0.96,4.8,12.7


## Data Splitting

In [3]:
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['bg+1:00'])
y = train_data['bg+1:00']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardization

In [4]:
from pipelines import standardization_pipeline

X_train = standardization_pipeline.fit_transform(X_train)
X_test = standardization_pipeline.transform(X_test)

X_train.head()

,bg-2:00,bg-1:55,bg-1:50,bg-1:45,bg-1:40,bg-1:35,bg-1:30,bg-1:25,bg-1:20,bg-1:15,...,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,day_phase_evening,day_phase_morning,day_phase_night,day_phase_noon
id,,,,,,,,,,,,,,,,,,,,,
p12_24040,-0.927655,-0.861759,-0.827864,-0.760280,-0.727643,-0.626891,-0.592867,-0.593618,-0.526773,-0.526568,...,-0.214903,-0.232818,-0.221298,-0.217413,-0.211511,-0.201556,0,0,0,1
p12_13267,-0.359878,-0.360222,-0.359971,-0.326209,-0.293123,-0.259468,-0.259266,-0.192837,-0.192847,-0.159615,...,-0.214903,-0.232818,-0.221298,-0.217413,-0.000161,-0.201556,0,1,0,0
p03_131,2.178421,2.147466,2.213443,2.244831,2.280574,2.279087,2.242744,2.312048,2.244819,2.142186,...,-0.859023,-0.842894,-0.858737,-0.853195,-0.854637,-0.852188,0,0,0,0
p12_20861,1.410251,1.345006,1.077130,0.508544,0.208246,-0.125860,-0.426067,-0.460024,-0.293025,-0.159615,...,-0.003227,1.764690,0.963619,2.626939,-0.193358,0.652399,0,0,0,0
p12_24627,-0.627067,-0.393658,-0.326550,-0.359599,-0.326548,-0.259468,-0.159186,-0.259634,-0.293025,-0.359771,...,-0.020109,-0.147948,0.372446,0.842650,0.632593,0.066043,0,0,0,0


## Model Selection with LazyPredict

In [5]:
from notebooks.helpers.LazyPredict import get_lazy_regressor

reg = get_lazy_regressor(exclude=['SVR'])
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 37/38 [27:22<00:10, 10.28s/it]   

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.052241 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31842
[LightGBM] [Info] Number of data points in the train set: 141619, number of used features: 129
[LightGBM] [Info] Start training from score 8.276012


100%|██████████| 38/38 [27:28<00:00, 43.39s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.67,0.68,1.71,433.87
BaggingRegressor,0.61,0.62,1.86,133.53
XGBRegressor,0.61,0.61,1.87,4.38
LGBMRegressor,0.59,0.59,1.93,6.22
HistGradientBoostingRegressor,0.58,0.59,1.94,12.74
GradientBoostingRegressor,0.54,0.54,2.04,229.60
KNeighborsRegressor,0.53,0.53,2.05,7.59
MLPRegressor,0.53,0.53,2.06,82.37
LassoLarsIC,0.50,0.50,2.12,1.58


The best models are:

* KNeighborsRegressor
* ExtraTreesRegressor
* BaggingRegressor

For further work I choose:

* KNeighborsRegressor
* BaggingRegressor
* XGBRegressor.